In [ ]:
import sys
print(sys.path)

In [ ]:
import dotenv
from fastbook import *
from fastai.vision.utils import *

In [ ]:
# expecting AZURE_SEARCH_KEY=<key> in .env file
dotenv.load_dotenv('.env')
key = os.environ.get('AZURE_SEARCH_KEY', 'XXX')
print('successfully fetched env vars')

In [ ]:
comic_collections_dict = {
    'the far side': 'tfs',
    'xkcd': 'xk',
    'dilbert': 'dbt',
    'pearls before swine': 'pbs',
    'rhymes with orange': 'rwo',
    'hägar the horrible': 'hth',
    'in the bleachers': 'itb'
}

comic_collections = comic_collections_dict.keys()

path = Path('comics')

In [ ]:
if path.exists():
    print(f'{os.path.abspath(path)} exists. Skipping downloading of images.')
else:
    path.mkdir()
    for name, abbr in comic_collections_dict.items():
        print(f'searching for "{name}" comics...')
        dest = (path/abbr)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{name} comics')
        print(f'downloading "{name}" comics...')
        download_images(dest, urls=results.attrgot('contentUrl'))
        print('done.')

In [ ]:
files = get_image_files(path)
print(f'{len(files)} attempted downloads')

In [ ]:
failed = verify_images(files)
print(f'{len(failed)} failed downloads')

In [ ]:
print('cleaning up failed downloads...')
failed.map(Path.unlink);
print('done.')

In [ ]:
comics = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
dls = comics.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=40, nrows=5)

In [ ]:
comics = comics.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = comics.dataloaders(path)
dls.train.show_batch(max_n=40, nrows=5, unique=True)

In [ ]:
comics = comics.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = comics.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')